In [1]:
#baseline model UNET
# I splitted every image to patches of 64 * 64 pixels 
#and classified every pixel in every patch as binary classification  task


#so for the model the input is 64 * 64 pixels images.


#model: UNET
#loss Binary cross entropy 
#optimizer: SGD

#I splitted to dataset under to dicom-images-train

In [2]:
!pip install pydicom

     |████████████████████████████████| 2.0 MB 4.6 MB/s 


In [66]:
from torch.utils.data import Dataset, WeightedRandomSampler
from skimage import io, transform
import os
import pandas as pd
from PIL import Image
import random
import numpy as np
from torchvision import transforms, datasets
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from imageio import imread
import torch
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import cm

import pydicom
from abc import ABC, abstractmethod
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation

In [4]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)


In [5]:

print('__pyTorch VERSION:', torch.__version__)
use_cuda = torch.cuda.is_available()
print('use_cuda = {0}'.format(use_cuda))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

DATA_FOLDER ='data'
device

__pyTorch VERSION: 1.11.0+cu113
use_cuda = True


'cuda'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls "/content/drive/My Drive/medical"


dicom-images-train.zip	train-rle.csv


In [9]:
!cp "/content/drive/My Drive/medical/dicom-images-train.zip" "dicom-images-train.zip"


In [10]:
!cp '/content/drive/My Drive/medical/train-rle.csv' "train-rle.csv"

In [11]:
!ls

dicom-images-train.zip	drive  sample_data  train-rle.csv


In [12]:
!unzip dicom-images-train.zip -d data

Streaming output truncated to the last 5000 lines.
   creating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2333.1517875172.216482/1.2.276.0.7230010.3.1.3.8323329.2333.1517875172.216481/
  inflating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.2333.1517875172.216482/1.2.276.0.7230010.3.1.3.8323329.2333.1517875172.216481/1.2.276.0.7230010.3.1.4.8323329.2333.1517875172.216483.dcm  
   creating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3857.1517875179.826165/
   creating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3857.1517875179.826165/1.2.276.0.7230010.3.1.3.8323329.3857.1517875179.826164/
  inflating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3857.1517875179.826165/1.2.276.0.7230010.3.1.3.8323329.3857.1517875179.826164/1.2.276.0.7230010.3.1.4.8323329.3857.1517875179.826166.dcm  
   creating: data/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.4161.1517875181.439748/
   creating: data/dicom-images-train/1.2.276.0.7230

In [13]:
SEED = 999

def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

fixSeed(SEED)
SEED

999

In [16]:
# df = pd.read_csv('archive/train-rle.csv', header=None, index_col=0)
# train_fns = sorted(glob.glob('archive/dicom-images-train/*/*/*.dcm'))#[0:500]
# df_full = pd.read_csv('archive/train-rle.csv', index_col='ImageId')

df = pd.read_csv('train-rle.csv', header=None, index_col=0)
train_fns = sorted(glob.glob('data/dicom-images-train/*/*/*.dcm'))
len(train_fns)
df_full = pd.read_csv('train-rle.csv', index_col='ImageId')
df_full.sample(1)

,EncodedPixels
ImageId,
1.2.276.0.7230010.3.1.4.8323329.32218.1517875159.68275,176473 3 1016 6 1013 9 997 7 6 11 996 26 995 ...


In [17]:
dataset_df = df_full.reset_index().drop_duplicates(['ImageId'])


negative_images =  set(dataset_df[dataset_df[' EncodedPixels'].str.contains('-1')].sample(330)['ImageId'].tolist())

positive_images = set(dataset_df[~dataset_df[' EncodedPixels'].str.contains('-1')].sample(330)['ImageId'].tolist())

images_ids_for_trainning = set.union(negative_images, positive_images)
len(images_ids_for_trainning)

660

In [18]:
im_height = 1024
im_width = 1024
im_chan = 1
valid_indexes = []
X_s = np.zeros((len(images_ids_for_trainning), im_height, im_width, im_chan), dtype=np.uint8)
Y_s = np.zeros((len(images_ids_for_trainning), im_height, im_width, 1), dtype=bool)
sys.stdout.flush()
index_to_set = 0

for _id in tqdm(train_fns, total=len(train_fns)):

    
    image_id = _id.split('/')[-1].split('.dcm')[0]

    
    if image_id not in images_ids_for_trainning:
      continue


    dataset = pydicom.read_file(_id)
    X_s[index_to_set] = np.expand_dims(dataset.pixel_array, axis=2)

  


    try:
        if '-1' in df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']:
            Y_s[index_to_set] = np.zeros((1024, 1024, 1))
        else:
            if type(df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']) == str:
                Y_s[index_to_set] = np.expand_dims(rle2mask(df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels'], 1024, 1024), axis=2)
            else:
                Y_s[index_to_set] = np.zeros((1024, 1024, 1))
                for x in df_full.loc[_id.split('/')[-1][:-4],' EncodedPixels']:
                    Y_s[index_to_set] =  Y_s[index_to_set] + np.expand_dims(rle2mask(x, 1024, 1024), axis=2)
    except KeyError:
        print(f"Key {_id.split('/')[-1][:-4]} without mask, assuming healthy patient.")
        Y_s[index_to_set] = np.zeros((1024, 1024, 1)) # Assume missing masks are empty masks.
    
    index_to_set += 1
  


100%|██████████| 10712/10712 [00:10<00:00, 1047.86it/s]


In [19]:
index_to_set

660

In [22]:
validation_size = int(Y_s.shape[0] * 0.1)
validation_indexes = np.random.choice(range(0, Y_s.shape[0]), size=validation_size, replace=False)
train_indexes  = [i for i in range(0, Y_s.shape[0]) if i not in validation_indexes]


y_train = Y_s[train_indexes, :, :]
y_validation = Y_s[validation_indexes, :, :]



X_validation = X_s[validation_indexes, :, :]
X_train = X_s[train_indexes, :, :]

y_train.shape, y_validation.shape

((594, 1024, 1024, 1), (66, 1024, 1024, 1))

In [23]:
im_height = 64
im_width = 64
X_train = X_train.reshape((-1, im_height, im_width, 1))
y_train = y_train.reshape((-1, im_height, im_width, 1))


X_validation = X_validation.reshape((-1, im_height, im_width, 1))
y_validation = y_validation.reshape((-1, im_height, im_width, 1))

X_train.shape, y_train.shape

((152064, 64, 64, 1), (152064, 64, 64, 1))

In [24]:
num_of_positive_pixels = np.sum(y_train)
num_of_negatuve_pixels = y_train.size - num_of_positive_pixels 

num_of_positive_pixels, num_of_negatuve_pixels 

(4185312, 618668832)

In [25]:
class PreProcessingStep(ABC):
    
    @abstractmethod
    def preprocess(image):
        pass

    
class PreProcessingWindow(PreProcessingStep):
    
    def __init__(self, preprocess_window, output_range):
        self._preprocess_window = preprocess_window 
        self._output_range = output_range
    
    def preprocess(self, image):
        windowed_image = np.interp(image, self._preprocess_window, self._output_range)
        return windowed_image


class PreProcessingAugmentations(PreProcessingStep):
    
    def __init__(self, augmentations):
        self.augmentations = augmentations 
    
    def preprocess(self, image):
      
      image =Image.fromarray(image.squeeze(2))

      for aug in self.augmentations:
        image = aug(image)

      return np.expand_dims(np.asarray(image), 2) 

class PreProcessingNormalization(PreProcessingStep):
    
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std 
    
    def preprocess(self, image):
      


      return (image  - self.mean) / self.std




mean, std = X_train.mean(), X_train.std()

preprocessing_normalization = PreProcessingNormalization(mean, std)


In [26]:
class DataGenerator(Dataset):
    def __init__(self, images, masks, preprocesing = None):
        self.images = images
        self.masks = masks
        self.preprocesing = preprocesing
        
      
    def __len__(self):
        return self.masks.shape[0]

    def __getitem__(self, index):
        
        y = self.masks[index]
        y = np.where(y == True, 1, 0)
        
        image  = X_train[index]
        
        if self.preprocesing is not None:
            for pre_processor in self.preprocesing:
                
                image = pre_processor.preprocess(image)
                
        
        image = image.reshape(1, image.shape[0], image.shape[1])
        y = y.reshape(1, y.shape[0], y.shape[1])
        
        return image, y

In [27]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [110]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [98]:
def train(model, data_loader, optimizer, criterion):
    calc_loss = 0.
    calc_count = 0.

    model.train()

    for images, labels in data_loader:

        calc_count += labels.shape[0]
        images = images.float().to(device)
     
        labels = labels.float().to(device)
        images = Variable(images)
        labels = Variable(labels)

        output = model(images)
        loss = criterion(output, labels)
        calc_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return calc_loss/calc_count


def eval_model(model, data_loader, criterion):
    model.eval()
    
    calc_loss = 0.
    calc_count = 0.
    
    with torch.no_grad():
        for images, labels in data_loader:
            calc_count += labels.shape[0]
            images = images.float().to(device)

            labels = labels.float().to(device)
            images = Variable(images)
            labels = Variable(labels)
            output = model(images)
            loss = criterion(output, labels)
            calc_loss += loss.item()
            
        
    return calc_loss/calc_count

    



In [108]:
def run_experiment(model, criterion, optimizer, numer_of_epoch, train_data_loader, validation_data_loader):
    
    for epoch in range(numer_of_epoch):
        print(epoch)
        train_loss = train(model, train_data_loader, optimizer, criterion)
        val_loss = eval_model(model, validation_data_loader, criterion)

        print('epoch {} train loss {} val loss {}'.format(epoch, train_loss, val_loss))


In [ ]:
#baseline
model = UNet(1, 1, False).to(device)
lr = 0.001
batch_size = 16
num_epochs = 50
momentum = 0.9

criterion =nn.BCEWithLogitsLoss().to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
training_generator = DataGenerator(X_train, y_train)
validation_generator = DataGenerator(X_validation, y_validation)

train_data_loader = DataLoader(training_generator, batch_size=8, shuffle=True, num_workers=0)
validation_data_loader = DataLoader(validation_generator, batch_size=8, shuffle=True, num_workers=0)
    

#run_experiment(model, criterion, optimizer, 100, train_data_loader, validation_data_loader)

In [ ]:
#weighted loss 

pos_weight = num_of_negatuve_pixels/num_of_positive_pixels 
pos_weight = torch.as_tensor(pos_weight, dtype=torch.float).to(device)



model = UNet(1, 1, False).to(device)
lr = 0.001
batch_size = 16
num_epochs = 2
momentum = 0.9

criterion =nn.BCEWithLogitsLoss(pos_weight = pos_weight).to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
training_generator = DataGenerator(X_train, y_train)
validation_generator = DataGenerator(X_validation, y_validation)

train_data_loader = DataLoader(training_generator, batch_size=8, shuffle=True, num_workers=0)
validation_data_loader = DataLoader(validation_generator, batch_size=8, shuffle=True, num_workers=0)
    

run_experiment(model, criterion, optimizer, num_epochs, train_data_loader, validation_data_loader)

In [ ]:
#weighted loss with image preprocessing

pos_weight = num_of_negatuve_pixels/num_of_positive_pixels 
pos_weight = torch.as_tensor(pos_weight, dtype=torch.float).to(device)

model = UNet(1, 1, False).to(device)
lr = 0.001
batch_size = 16
num_epochs = 50
momentum = 0.9

criterion =nn.BCEWithLogitsLoss(pos_weight = pos_weight).to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

preprocesing = [preprocessing_normalization]

training_generator = DataGenerator(X_train, y_train, preprocesing)
validation_generator = DataGenerator(X_validation, y_validation, preprocesing)

train_data_loader = DataLoader(training_generator, batch_size=8, shuffle=True, num_workers=0)
validation_data_loader = DataLoader(validation_generator, batch_size=8, shuffle=True, num_workers=0)
    

run_experiment(model, criterion, optimizer, 100, train_data_loader, validation_data_loader)


In [ ]:
#weighted loss with image preprocessing and augmentations

pos_weight = num_of_negatuve_pixels/num_of_positive_pixels 
pos_weight = torch.as_tensor(pos_weight, dtype=torch.float).to(device)

model = UNet(1, 1, False).to(device)
lr = 0.001
batch_size = 16
num_epochs = 50
momentum = 0.9
number_of_epocs = 2

criterion =nn.BCEWithLogitsLoss(pos_weight = pos_weight).to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

aug_preprocessing = PreProcessingAugmentations([RandomHorizontalFlip(0.5), RandomVerticalFlip(0.5), RandomRotation(0.3)])

preprocesing = [preprocessing_normalization, aug_preprocessing]

training_generator = DataGenerator(X_train, y_train, preprocesing)
validation_generator = DataGenerator(X_validation, y_validation, preprocesing)

train_data_loader = DataLoader(training_generator, batch_size=8, shuffle=True, num_workers=0)
validation_data_loader = DataLoader(validation_generator, batch_size=8, shuffle=True, num_workers=0)
    

run_experiment(model, criterion, optimizer, number_of_epocs, train_data_loader, validation_data_loader)


In [60]:
# y_train.shape, 
slices_positive_counts = np.sum(y_train, axis=(1,2,3))
positive_slices = set(np.where(x > 0)[0])

total_slices = float(y_train.shape[0])
num_of_negative_slices = y_train.shape[0] - len(positive_slices)
num_of_positive_slices = len(positive_slices)

weights = [1.0/num_of_positive_slices  if  i in positive_slices else 1.0/num_of_negative_slices for i in range(y_train.shape[0])]

In [82]:
#sampling positive and negative slices ~equally using WeightedRandomSampler

model = UNet(1, 1, False).to(device)
lr = 0.0001
batch_size = 16
num_epochs = 50
momentum = 0.9
number_of_epocs = 50

criterion =nn.BCEWithLogitsLoss().to(device) 

#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


aug_preprocessing = PreProcessingAugmentations([RandomHorizontalFlip(0.5), RandomVerticalFlip(0.5), RandomRotation(0.3)])

preprocesing = [preprocessing_normalization, aug_preprocessing]

training_generator = DataGenerator(X_train, y_train, preprocesing)
validation_generator = DataGenerator(X_validation, y_validation, preprocesing)

train_data_loader = DataLoader(training_generator, batch_size=batch_size, num_workers=0, sampler = WeightedRandomSampler(weights, batch_size, replacement=False))
validation_data_loader = DataLoader(validation_generator, batch_size=batch_size, shuffle=True, num_workers=0)
    

run_experiment(model, criterion, optimizer, number_of_epocs, train_data_loader, validation_data_loader)

In [ ]:
#sampling positive and negative slices ~equally using WeightedRandomSampler
# loss = DiceBCELoss

model = UNet(1, 1, False).to(device)
lr = 0.0001
batch_size = 16
num_epochs = 50
momentum = 0.9
number_of_epocs = 50

criterion = DiceBCELoss().to(device) 

#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


aug_preprocessing = PreProcessingAugmentations([RandomHorizontalFlip(0.5), RandomVerticalFlip(0.5), RandomRotation(0.3)])

preprocesing = [preprocessing_normalization, aug_preprocessing]

training_generator = DataGenerator(X_train, y_train, preprocesing)
validation_generator = DataGenerator(X_validation, y_validation, preprocesing)

train_data_loader = DataLoader(training_generator, batch_size=batch_size, num_workers=0, sampler = WeightedRandomSampler(weights, batch_size, replacement=False))
validation_data_loader = DataLoader(validation_generator, batch_size=batch_size, shuffle=True, num_workers=0)
    

run_experiment(model, criterion, optimizer, number_of_epocs, train_data_loader, validation_data_loader)

0
epoch 0 train loss 0.11822892725467682 val loss 0.10621098280094113
1
epoch 1 train loss 0.1183224618434906 val loss 0.10638597712033626
2
epoch 2 train loss 0.11690132319927216 val loss 0.10663927638564598
3
